# Exercice n°4

Dans un premier temps, il faut s'assurer que l'overlay est bien installé

In [19]:
from pynq import Overlay

overlay = Overlay("/home/xilinx/jupyter_notebooks/SE_EN325/S7_8/fft.bit")
#overlay = Overlay("/home/xilinx/jupyter_notebooks/SE_EN325/S6_Ex2/Axi_Stream_Lite.bit")
overlay?

Configuration du bloc FIR8_Stream issu de la synthèse de haut niveau

In [20]:
FFT8_Stream_IP = overlay.FFT8_Stream_0
#FIR8_Stream_IP = overlay.Axi_Stream_Lite_0
FFT8_Stream_IP?
#BASE_ADDRESS = 0x40000000
BASE_ADDRESS = overlay.ip_dict['FFT8_Stream_0']['phys_addr']
#BASE_ADDRESS = overlay.ip_dict['Axi_Stream_Lite_0']['phys_addr']
ADDRESS_LENGTH = 0x64000
print("Physical address of MAC IP : 0x" + format(BASE_ADDRESS, '02x'))

Physical address of MAC IP : 0x40000000


Configuration du bloc DMA ajouté sous Vivado

In [21]:
import pynq.lib.dma
DMA = overlay.axi_dma_0 
#DMA = overlay.axi_dma_0
DMA_send = DMA.sendchannel
DMA_recv = DMA.recvchannel
DMA?

Vérification de l'état de l'IP FIR8_Stream

In [22]:
FFT8_Stream_IP.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

Lancement de l'IP FIR8_Stream

In [23]:
FFT8_Stream_IP.write (0x0,0b10000001) # Activation des bits 0 (ap_start) et 7 (auto_restart)
#FIR8_Stream_IP.register_map.CTRL.AP_START=1
FFT8_Stream_IP.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

Vérification de l'état de l'IP DMA

In [24]:
DMA.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=1, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

# Ecrire un programme Python pour l’exécution suivante :


-- Allouer 2 buffers de taille 32: (input_buffer  et output_buffer)

-- Fixer les valeurs 0 à 19 à input_buffer

-- Effectuer l'envoi puis la récéption de la trame de taille 20

-- Afficher le contenu de output_buffer 

-- Libérer l'espace mémoire allouée aux deux buffers



In [25]:
from pynq import allocate
import numpy as np

# -------------------------
# Nombre de samples
# -------------------------
SAMPLES = 40  # tu as 40 samples

# -------------------------
# Allocation des buffers
# -------------------------
# Buffer interleavé : real0, imag0, real1, imag1, ...
input_buffer = allocate(shape=(2*SAMPLES,), dtype=np.int32)
output_buffer = allocate(shape=(2*SAMPLES,), dtype=np.int32)

# -------------------------
# Lecture des fichiers
# -------------------------
with open("/home/xilinx/jupyter_notebooks/SE_EN325/S7_8/in_real.txt") as f_re, \
     open("/home/xilinx/jupyter_notebooks/SE_EN325/S7_8/in_imag.txt") as f_im:

    real_vals = [int(x.strip()) for x in f_re]
    imag_vals = [int(x.strip()) for x in f_im]

# Vérification
assert len(real_vals) == SAMPLES, f"Nombre de lignes dans in_real.txt incorrect : {len(real_vals)}"
assert len(imag_vals) == SAMPLES, f"Nombre de lignes dans in_imag.txt incorrect : {len(imag_vals)}"

# -------------------------
# Création de la trame AXI-Stream interleavée
# -------------------------
for i in range(SAMPLES):
    input_buffer[2*i]   = real_vals[i]  # indices pairs -> réels
    input_buffer[2*i+1] = imag_vals[i]  # indices impairs -> imaginaires

# -------------------------
# Transfert vers l'IP FFT via DMA
# -------------------------
# DMA_send : transfert complet
DMA_send.transfer(input_buffer)
DMA_send.wait()  # attendre que tout soit envoyé

# DMA_recv : réception complète
DMA_recv.transfer(output_buffer)
DMA_recv.wait()  # attendre que tout soit reçu

# -------------------------
# Extraction des résultats réels et imaginaires
# -------------------------
output_real = output_buffer[0::2]  # indices pairs
output_imag = output_buffer[1::2]  # indices impairs

# -------------------------
# Affichage des résultats
# -------------------------
for i in range(SAMPLES):
    print(f"out_real[{i}] = {output_real[i]}")
    print(f"out_imag[{i}] = {output_imag[i]}")

# -------------------------
# Cleanup
# -------------------------
del input_buffer, output_buffer


KeyboardInterrupt: 

In [26]:
from pynq import allocate
import numpy as np

data_size = 40
input_buffer_im = allocate(shape=(data_size,), dtype=np.int32)
input_buffer_re = allocate(shape=(data_size,), dtype=np.int32)
output_buffer_im = allocate(shape=(data_size,), dtype=np.int32)
output_buffer_re = allocate(shape=(data_size,), dtype=np.int32)

file_data_im = open("/home/xilinx/jupyter_notebooks/SE_EN325/S7_8/in_imag.txt", "r")
file_data_re = open("/home/xilinx/jupyter_notebooks/SE_EN325/S7_8/in_real.txt", "r")
i=0
for line in file_data_im:
    val = line.strip(',\n')
    val_int = int(val)
    input_buffer_im[i] = int(val_int)
    print("La Valeur de input_buffer[",i,"] =", input_buffer_im[i])
    i=i+1
file_data_im.close()

i=0
for line in file_data_re:
    val = line.strip(',\n')
    val_int = int(val)
    input_buffer_re[i] = int(val_int)
    print("La Valeur de input_buffer[",i,"] =", input_buffer_re[i])
    i=i+1
file_data_re.close()


DMA_send.transfer(input_buffer_re)
DMA_send.wait()
DMA_send.transfer(input_buffer_im)
DMA_send.wait()

print("----------------------------------")
print("----------------------------------")

DMA_recv.transfer(output_buffer_re)
DMA_recv.wait()
DMA_recv.transfer(output_buffer_im)
DMA_recv.wait()

for i in range(data_size):
    print("La Valeur de output_buffer_re[",i,"] =", output_buffer_re[i])
    print("La Valeur de output_buffer_im[",i,"] =", output_buffer_im[i])

del input_buffer_re, output_buffer_re
del input_buffer_im, output_buffer_im

La Valeur de input_buffer[ 0 ] = 2011
La Valeur de input_buffer[ 1 ] = 2011
La Valeur de input_buffer[ 2 ] = 2011
La Valeur de input_buffer[ 3 ] = 2011
La Valeur de input_buffer[ 4 ] = 2011
La Valeur de input_buffer[ 5 ] = 2011
La Valeur de input_buffer[ 6 ] = 2011
La Valeur de input_buffer[ 7 ] = 2011
La Valeur de input_buffer[ 8 ] = 0
La Valeur de input_buffer[ 9 ] = 0
La Valeur de input_buffer[ 10 ] = 0
La Valeur de input_buffer[ 11 ] = 0
La Valeur de input_buffer[ 12 ] = 0
La Valeur de input_buffer[ 13 ] = 0
La Valeur de input_buffer[ 14 ] = 0
La Valeur de input_buffer[ 15 ] = 1
La Valeur de input_buffer[ 16 ] = -768
La Valeur de input_buffer[ 17 ] = -456
La Valeur de input_buffer[ 18 ] = -234
La Valeur de input_buffer[ 19 ] = -120
La Valeur de input_buffer[ 20 ] = 0
La Valeur de input_buffer[ 21 ] = 120
La Valeur de input_buffer[ 22 ] = 234
La Valeur de input_buffer[ 23 ] = 456
La Valeur de input_buffer[ 24 ] = 422
La Valeur de input_buffer[ 25 ] = 823
La Valeur de input_buffer[ 2

RuntimeError: DMA channel not halted

In [27]:
DMA.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=1, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=377786368),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=128),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSl